# Batch GIF Processing Notebook

This notebook provides functionality to:
1. Process multiple GIFs from an input folder
2. Resize all GIFs to specified square dimensions
3. Add text overlay with the GIF filename and standard text
4. Add logo overlay at selected positions
5. Compress the resulting GIFs


## 1. Import of Libraries

In [ ]:
import re
from PIL import Image, ImageDraw, ImageFont
import os
from IPython.display import display, Image as IPImage
import numpy as np


In [ ]:
import os
import shutil
import re

def copy_and_clean_gifs(source_folder, target_folder="input"):
    """
    Copy GIF files from source folder to target folder,
    removing numbered duplicates (e.g., removes 'xxx1.gif' if 'xxx.gif' exists)

    Args:
        source_folder (str): Path to source folder containing GIFs
        target_folder (str): Path to target folder (defaults to 'input')
    """


    # Ensure target folder exists
    os.makedirs(target_folder, exist_ok=True)

    # Get all GIF files from source
    gif_files = [f for f in os.listdir(source_folder) if f.lower().endswith('.gif')]

    # Dictionary to store base names and their variations
    file_groups = {}

    # Group files by their base names
    for file in gif_files:
        # Remove .gif extension
        base = file[:-4]
        # Check if the filename ends with a number
        match = re.match(r'(.*?)\d+$', base)

        if match:
            # If it has a number, use the part before the number as key
            key = match.group(1).rstrip()
        else:
            # If no number, use the whole base as key
            key = base

        if key not in file_groups:
            file_groups[key] = []
        file_groups[key].append(file)

    # Copy files, skipping numbered versions if base version exists
    copied_count = 0
    skipped_count = 0

    for base_name, variations in file_groups.items():
        # Sort variations to ensure base version (without number) comes first if it exists
        variations.sort(key=lambda x: (len(x), x))

        # Copy the first variation (usually the base version)
        source_path = os.path.join(source_folder, variations[0])
        target_path = os.path.join(target_folder, variations[0])
        shutil.copy2(source_path, target_path)
        copied_count += 1

        # Count skipped variations
        skipped_count += len(variations) - 1

    print(f"Copied {copied_count} files")
    if skipped_count > 0:
        print(f"Skipped {skipped_count} numbered variations")

# Example usage:
# copy_and_clean_gifs("/path/to/source/folder")
# Or with custom target: copy_and_clean_gifs("/path/to/source", "custom_input")

## 2. Input/Output Settings

In [ ]:
# Directory settings
INPUT_DIR = "input"
OUTPUT_DIR = "output"
ASSETS_DIR = "assets"

current_dir = os.getcwd()
print(current_dir)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
print(parent_dir)
copy_and_clean_gifs(parent_dir, INPUT_DIR)

# Create directories if they don't exist
for directory in [INPUT_DIR, OUTPUT_DIR, ASSETS_DIR]:
    os.makedirs(directory, exist_ok=True)

# GIF Settings
GIF_SIZE = 600  # Size for square output
QUALITY = 20  # GIF quality (1-100)

# Logo Settings
LOGO_FILE = "logo.png"  # Place your logo in files/assets/
LOGO_POSITION = (15, 10)

# Text Overlay Settings
FONT_SIZE = 16
WHITE_COLOR = (255, 255, 255)
BLACK_COLOR = (0, 0, 0)
FONT_NAME = "lucida-grande"

def create_text_overlays(filename):
    """Create text overlays using the GIF filename as text_1"""
    # Clean up filename by removing extension and replacing underscores/hyphens with spaces
    clean_name = os.path.splitext(filename)[0].replace('_', ' ').replace('-', ' ')

    return [
        {
            "text": clean_name,
            "position": (10, GIF_SIZE - 10 - FONT_SIZE),  # Bottom left
            "font": FONT_NAME,
            "color": WHITE_COLOR,
            "stroke_width": 2,
            "stroke_fill": BLACK_COLOR
        },
        {
            "text": "Available in our materials bank",
            "position": (GIF_SIZE//2 + 50, GIF_SIZE - 10 - FONT_SIZE),  # Bottom right
            "font": FONT_NAME,
            "color": WHITE_COLOR,
            "stroke_width": 2,
            "stroke_fill": BLACK_COLOR
        }
    ]


## 3. Font Manager Class

In [ ]:
import matplotlib.font_manager as fm

class FontManager:
    """Manages fonts for the GIF processor"""

    def __init__(self):
        """Initialize font manager and discover available fonts"""
        self.fonts = self._discover_fonts()

    def _discover_fonts(self):
        """Discover all available fonts in the system"""
        fonts = {}
        for font in fm.fontManager.ttflist:
            try:
                name = font.name.lower().replace(' ', '-')
                fonts[name] = font.fname
            except Exception:
                continue
        return fonts

    def get_font(self, font_name='default', size=30):
        """Get a font by name and size"""
        try:
            if font_name == 'default':
                return ImageFont.load_default()

            if font_name in self.fonts:
                return ImageFont.truetype(self.fonts[font_name], size)

            fuzzy_matches = [path for name, path in self.fonts.items()
                           if font_name in name and name != 'default']
            if fuzzy_matches:
                return ImageFont.truetype(fuzzy_matches[0], size)

            raise ValueError(f"Font '{font_name}' not found")

        except Exception as e:
            print(f"Error loading font {font_name}: {str(e)}")
            return ImageFont.load_default()

    def list_fonts(self):
        """List all available fonts"""
        return ['default'] + sorted(list(self.fonts.keys()))

    def search_fonts(self, query):
        """Search for fonts containing the query string"""
        query = query.lower()
        matches = [name for name in self.fonts.keys() if query in name]
        return sorted(matches)


## 3.2. List fonts

In [ ]:
# Initialize font manager and list available fonts
font_manager = FontManager()
print("Available fonts:")
print(font_manager.list_fonts())


## 4. GIF Processor Class

In [ ]:
from io import BytesIO

class GIFProcessor:
    def __init__(self, gif_path):
        """Initialize with path to GIF file"""
        self.gif = Image.open(gif_path)
        self.frames = []
        self.durations = []

        # Extract all frames and their durations
        try:
            while True:
                self.frames.append(self.gif.copy())
                self.durations.append(self.gif.info.get('duration', 100))
                self.gif.seek(self.gif.tell() + 1)
        except EOFError:
            pass

    def resize(self, width, height):
        """Resize all frames to specified dimensions"""
        self.frames = [frame.resize((width, height), Image.Resampling.LANCZOS)
                      for frame in self.frames]
        return self

    def make_square(self, size=None):
        """
        Crop the GIF to a square from the center.
        If size is provided, the output will be resized to size x size.
        If size is None, the square will be sized to the smaller dimension.
        """
        if not self.frames:
            return self

        # Get dimensions from first frame
        width, height = self.frames[0].size

        # Calculate crop box for square
        if width > height:
            # Landscape orientation
            left = (width - height) // 2
            top = 0
            right = left + height
            bottom = height
        else:
            # Portrait orientation
            left = 0
            top = (height - width) // 2
            right = width
            bottom = top + width

        # Apply crop to all frames
        self.frames = [frame.crop((left, top, right, bottom)) for frame in self.frames]

        # Resize if size is specified
        if size is not None:
            self.frames = [frame.resize((size, size), Image.Resampling.LANCZOS)
                         for frame in self.frames]

        return self

    def add_text(self, text, position, font_path=None, font_size=30,
                 color=(255, 255, 255), stroke_width=2, stroke_fill=(0, 0, 0)):
        """Add text overlay to all frames"""
        font_manager = FontManager()
        font = font_manager.get_font(font_name=font_path,size=font_size)

        for i, frame in enumerate(self.frames):
            # Convert to RGBA before drawing
            frame_rgba = frame.convert('RGBA')
            draw = ImageDraw.Draw(frame_rgba)
            draw.text(position, text, font=font, fill=color,
                     stroke_width=stroke_width, stroke_fill=stroke_fill)
            self.frames[i] = frame_rgba
        return self

    def add_image_overlay(self, overlay_path, position):
        """Add image overlay to all frames"""
        overlay = Image.open(overlay_path).convert('RGBA')

        for i, frame in enumerate(self.frames):
            frame_rgba = frame.convert('RGBA')
            frame_rgba.paste(overlay, position, overlay)
            self.frames[i] = frame_rgba
        return self

    def optimize(self, quality=100):
        if not self.frames:
            return

        # Convert frames to RGB mode for saving
        rgb_frames = []
        for frame in self.frames:
            rgb_frame = frame.convert('RGB')
            # Calculate number of colors based on quality
            n_colors = max(min(256, int(256 * (quality / 100))), 2)

            # Convert to P mode (palette) with optimized palette
            rgb_frame = rgb_frame.quantize(
                colors=n_colors,
                method=Image.Quantize.MEDIANCUT,
                dither=Image.Dither.FLOYDSTEINBERG
            )
            rgb_frames.append(rgb_frame)

        self.frames = rgb_frames

    def save(self, output_path, optimize=False, quality=100):
        """
        Save the processed GIF with optimization options

        Args:
            output_path (str): Path to save the GIF
            optimize (bool): Whether to optimize the GIF
            quality (int): Quality from 1 (worst) to 100 (best).
                         Lower quality means smaller file size.
        """
        if not self.frames:
            return
        if optimize:
            self.optimize(quality)
        rgb_frames = [frame.convert('RGB') for frame in self.frames]

        # Save with optimization
        rgb_frames[0].save(
            output_path,
            save_all=True,
            append_images=rgb_frames[1:],
            optimize=optimize,
            duration=self.durations,
            loop=0,
            format='GIF',
            # Additional optimization parameters
            disposal=2,  # Clear the frame before rendering the next
            quality=quality
        )
        print("Size on disk:", f"{os.path.getsize(output_path) / 1024 / 1024:.2f} MB")

    def display(self):
        """Display the current state of the GIF in the notebook"""
        temp_path = '_temp_display.gif'
        self.save(temp_path)
        display(IPImage(filename=temp_path))
        os.remove(temp_path)

    def get_size(self, optimize=False, quality=100):
        """Get the size of the processed GIF in bytes without saving to disk"""
        if self.frames:
            # Convert frames back to RGB mode for saving
            rgb_frames = [frame.convert('RGB') for frame in self.frames]
            with BytesIO() as buffer:
                rgb_frames[0].save(
                    buffer,
                    save_all=True,
                    append_images=rgb_frames[1:],
                    optimize=optimize,
                    quality=quality,
                    duration=self.durations,
                    loop=0,
                    format='GIF'
                )
                return buffer.tell()


## 5. Process All GIFs

In [ ]:
def process_all_gifs():
    """Process all GIFs in the input directory"""
    # Get logo path
    logo_path = os.path.join(ASSETS_DIR, LOGO_FILE)
    if not os.path.exists(logo_path):
        print(f"Warning: Logo file not found at {logo_path}")
        return

    # Get all GIF files from input directory
    gif_files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith('.gif')]

    if not gif_files:
        print("No GIF files found in input directory")
        return

    print(f"Found {len(gif_files)} GIF files to process")

    # Process each GIF
    for gif_file in gif_files:
        try:
            print(f"\nProcessing: {gif_file}")

            input_path = os.path.join(INPUT_DIR, gif_file)
            output_path = os.path.join(OUTPUT_DIR, f"processed_{gif_file}")

            # Create GIF processor
            gif_processor = GIFProcessor(input_path)

            # Make square and resize
            gif_processor.make_square(size=GIF_SIZE)

            # Add text overlays
            text_overlays = create_text_overlays(gif_file)
            for overlay in text_overlays:
                gif_processor.add_text(
                    text=overlay["text"],
                    position=overlay["position"],
                    font_path=overlay["font"],
                    font_size=FONT_SIZE,
                    color=overlay["color"],
                    stroke_width=overlay["stroke_width"],
                    stroke_fill=overlay["stroke_fill"]
                )

            # Add logo
            gif_processor.add_image_overlay(logo_path, position=LOGO_POSITION)

            # Optimize and save
            gif_processor.optimize(quality=QUALITY)
            gif_processor.save(output_path, optimize=False, quality=QUALITY)

            print(f"Successfully processed: {gif_file}")

        except Exception as e:
            print(f"Error processing {gif_file}: {str(e)}")
            continue

# Run the batch processing
process_all_gifs()